# Optimización de Conjuntos de Datos para ML - 100473223 - Juan Leal Aliaga

---

## 1. Modificaciones Planteadas

### 1.1 Síntesis de Atributos (Programación Genética)
- **Objetivo**: Crear 4 nuevas features mediante árboles de expresiones matemáticas
- **Operadores**: add, sub, mul, div, sqrt, square, log, sin, cos, tanh
- **Terminales**: Variables originales X₀...Xₙ y constantes [-5, 5]

### 1.2 Reducción de Dimensionalidad (Feature Selection)
- **Objetivo**: Seleccionar subconjunto óptimo de features (originales + generadas)
- **Método**: Algoritmo Genético con codificación binaria
- **Rango**: Entre 3 y 15 features seleccionadas

---

## 2. Técnicas de Computación Evolutiva

### 2.1 Programación Genética (GP)
**Población**: 100 individuos  
**Representación**: Lista de árboles  
**Profundidad**: 2-5 niveles  

**Operadores Genéticos**:
- Cruce a nivel de subárbol (prob. 0.8)
- Mutación de nodos/valores (prob. 0.2)
- Reemplazo aleatorio de árboles completos

### 2.2 Algoritmo Genético (GA) para Feature Selection
**Población**: 30 individuos  
**Representación**: Vector binario [1,0,1,1,0...]  

**Operadores Genéticos**:
- Cruce uniforme (prob. 0.8)
- Mutación por bit-flip 1-3 bits (prob. 0.3)
- Selección por torneo tamaño 3

---

## 3. Codificación, Operadores y Función de Fitness

### 3.1 Codificación en GP

- individuo = [árbol_1, árbol_2, árbol_3, árbol_4]
- Ejemplo de árbol: (X₀ + sqrt(X₁))

### 3.2 Función de Fitness (GP)

fitness = MSE_cv + λ · complejidad

Donde:
- MSE_cv: Error cuadrático medio en 3-fold cross-validation
- Complejidad: Suma de nodos de todos los árboles
- λ = 0.001, penalización por bloat (Excesiva expansión del árbol)

### 3.3 Función de Fitness (Feature Selection)
fitness = MSE_validation + 0.01 · n_features

Donde:
- MSE_validation: Error en conjunto de validación (20% train)
- n_features: Número de features seleccionadas
- Restricción: mínimo 2 features


---

## 4. Intentos Fallidos

### 4.1 Penalizaciones Excesivas
- Penalizaciones altas (λ=0.01) causaban árboles muy pequeños
- Resultados pobres en test set
### 4.2 Overfitting en GP
- Sin penalización, árboles crecían demasiado
- Buen rendimiento en train, malo en test
### 4.3 Selección de Features sin Restricción
- Individuos seleccionaban pocas features (1-2)
- Resultados inestables y pobres
### 4.4 Cambios de estrategia
- Diferentes funciones (pow, exp, trigonométricas...)
- Uso entre 3, 4, 5 modelos pero elección random
- Train/Validation en lugar de cv
- 
---

## 4. Resultados Obtenidos

### 4.1 Configuración Experimental
- **Modelos entrenados**: Uso Ensemble de Ridge y Random Forest tanto en GP como SE
- **Datasets**: California Housing (20,640 × 8) y Diabetes (442 × 10)
- **Tiempos**: 20 minutos y 1 hora
- **Modelos evaluados**: 16 (Ridge, LinearReg, RF, XGBoost, SVR, etc.)
- **Métricas**: MAE y MSE en test set (20%)

### 4.2 Hallazgos Principales

**Mejoras Consistentes**: 
- La mayoría de modelos mejoraron en ambos datasets
- Se intenta que funcione correctamente con modelos de todos los tipos: Lineales, ensembles,...

**Reducción Efectiva**: 
- De 8-10 features originales → 6-8 features optimizadas
- Manteniendo o mejorando el rendimiento

**Generalización**:
- Las transformaciones funcionan en múltiples tipos de modelos
- No sobreajuste: mejoras en test, no solo en train

**Actuación del Early-Stopping**:
- Para mi configuración final, el early-stopping no suele actuar ya que cada generación tarda bastante

---

## 5. Conclusiones

### 5.1 Estrategias Clave Implementadas
- **Early stopping** con validación interna (50/30 gens)
- **Protección robusta**: Clipping, NaN/Inf handling, división protegida
- **Regularización**: Ridge + RF
- **Evaluación robusta**: Cross-validation


### 5.2 Trabajo Futuro
1. **Paralelización**: Evaluación paralela de individuos con multiprocessing
2. **Multi-objetivo**: Optimizar simultáneamente MSE, MAE y complejidad (Pareto)
3. **Memoización**: Cachear evaluaciones de subárboles repetidos
4. **Transfer learning**: Reutilizar features entre datasets similares

### 5.3 Conclusión Final y Resultados (Benchmark)
Las mejoras consistentes obtenidas en múltiples modelos y datasets validan la efectividad del enfoque híbrido **GP + GA**, posicionándolo como una solución viable para preprocesamiento automático en proyectos reales de Machine Learning.

Aquí tienes tu texto en **formato Markdown** bien estructurado y legible:

---

## Resumen completo --> Todos los modelos con Ensemble

| Modelo           | MAE Base | MAE Opt | Mejora MAE   | MSE Base | MSE Opt | Mejora MSE   |
| ---------------- | -------- | ------- | ------------ | -------- | ------- | ------------ |
| KNeighbors       | 0.8128   | 0.3880  | **52.26 %**  | 1.1187   | 0.3386  | **69.73 %**  |
| MLP              | 0.6506   | 0.4080  | **37.28 %**  | 0.7129   | 0.3602  | **49.47 %**  |
| SVR              | 0.8600   | 0.5976  | **30.50 %**  | 1.3320   | 0.6877  | **48.37 %**  |
| HuberRegressor   | 0.5876   | 0.4889  | **16.79 %**  | 0.7119   | 0.4671  | **34.38 %**  |
| LinearRegression | 0.5332   | 0.4958  | **7.02 %**   | 0.5559   | 0.4569  | **17.82 %**  |
| Ridge            | 0.5332   | 0.4957  | **7.03 %**   | 0.5558   | 0.4569  | **17.80 %**  |
| BayesianRidge    | 0.5332   | 0.4958  | **7.02 %**   | 0.5556   | 0.4569  | **17.77 %**  |
| DecisionTree     | 0.4547   | 0.4168  | **8.34 %**   | 0.4952   | 0.4424  | **10.68 %**  |
| AdaBoost         | 0.6498   | 0.6086  | **6.34 %**   | 0.6145   | 0.5722  | **6.87 %**   |
| GradientBoosting | 0.3716   | 0.3587  | **3.48 %**   | 0.2940   | 0.2852  | **2.99 %**   |
| Bagging          | 0.3279   | 0.3158  | **3.68 %**   | 0.2559   | 0.2499  | **2.34 %**   |
| RandomForest     | 0.3275   | 0.3158  | **3.59 %**   | 0.2554   | 0.2495  | **2.30 %**   |
| ExtraTrees       | 0.3270   | 0.3207  | **1.93 %**   | 0.2539   | 0.2484  | **2.16 %**   |
| XGBoost          | 0.3096   | 0.3100  | **–0.12 %**  | 0.2226   | 0.2301  | **–3.35 %**  |
| Lasso            | 0.7616   | 0.9061  | **–18.97 %** | 0.9380   | 1.3107  | **–39.73 %** |
| ElasticNet       | 0.6763   | 0.8387  | **–24.02 %** | 0.7646   | 1.1262  | **–47.31 %** |

---

## ESTADÍSTICAS GENERALES CON ENSEMBLE

* **Modelos que mejoraron MAE:** 13 / 16
* **Modelos que mejoraron MSE:** 13 / 16
* **Mejor mejora MAE:** 52.26 % *(KNeighbors)*
* **Mejor mejora MSE:** 69.73 % *(KNeighbors)*
* **Mejora promedio MAE:** 8.88 %
* **Mejora promedio MSE:** 12.02 %
* **Evaluación con Ensemble:** *Ridge + RandomForest (promedio)*

---

## 6. Anexo de IA Generativa

### 6.1 Uso de IA Generativa en el Proyecto

Durante el desarrollo de este proyecto se utilizó **IA Generativa** (Claude, ChatGPT, Codex...) para:

#### Tareas Realizadas:
1. **Debugging de código**: Identificar errores en la evaluación de fitness
2. **Optimización de rendimiento**: Sugerencias para reducir tiempo de ejecución
3. **Exploración de hiperparámetros**: Recomendaciones de valores iniciales
4. **Generación de visualizaciones**: Código para gráficos de evolución del fitness

### 6.2 Ejemplos de Prompts Utilizados

#### Prompt 1: Debugging
```
"Tengo un error NaN en la evaluación de fitness de mi GP. 
El código es el siguiente: [...]. ¿Cómo puedo proteger contra 
divisiones por cero y valores infinitos?"
```

**Resultado obtenido**:
Solución implementada

#### Prompt 2: Optimización
```
"Mi algoritmo genético para feature selection es muy lento. 
Usa cross-validation 5-fold en cada evaluación. ¿Cómo puedo 
acelerar sin perder demasiada precisión?"
```

**Resultado obtenido**:
- Usar validación simple en lugar de CV durante la búsqueda
- Aplicar CV solo al mejor individuo final
- Reducir population_size de 50 a 30
- Implementar early stopping agresivo (30 generaciones)

#### Prompt 3: Estrategia de Reparto de Tiempo
```
"Tengo maxtime=3600 segundos. ¿Cómo debería repartir el tiempo 
entre la fase de GP (síntesis) y GA (selección) para maximizar 
la mejora final?"
```

**Resultado obtenido**:
- 70% del tiempo para GP (síntesis de features)
- 30% del tiempo para GA (selección)
- Justificación: la síntesis es más compleja y beneficia más

#### Prompt 4: Comparación de Modelos
```
"Tengo resultados de 17 modelos con/sin optimización. 
Dame código para crear una tabla ordenada por mejora en MSE"
```

**Resultado**: Código del bloque de análisis comparativo usado en el notebook

#### Prompt 5: Visualización de Convergencia
```
"Quiero graficar la evolución del mejor fitness a lo largo de 
las generaciones. ¿Cómo visualizo el progreso del GP?"
```

**Resultado**: 
Código base de plot...
